In [1]:
import geopandas
import pandas as pd
import glob
import shapefile
from tqdm import tqdm
import numpy as np
import pickle
import xlrd3

In [2]:
# read poi from shp files
shp_files = glob.glob('/home/yangwenhao/local/project/weibo_filter/app/data/tianjin/*.shp')

places_names = []
for f in tqdm(shp_files):
    x = geopandas.read_file(f, engine="pyogrio")
    places_names.append(x.name)
    
names = pd.concat(places_names, axis=0)

names.to_csv('tianjin.csv', index=False)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:38<00:00,  1.66s/it]


In [9]:
names = pd.read_csv('tianjin.csv').name

In [ ]:
greenbelt = 'greenbelt'
greenbelt_list = pd.read_excel('/home/yangwenhao/local/project/weibo_filter/app/data/' + greenbelt + '.xls')

In [86]:
names.head(2)

0    元记电气焊补胎钣金
1       元汇津通卸载
Name: name, dtype: object

In [16]:
location_names = greenbelt_list.name # names

# 'first_2w'
for lst in ['first_2w', 'noimg_6w']:
    data = xlrd3.open_workbook('/home/yangwenhao/local/project/weibo_filter/app/data/' + lst + '.xls')
    table = data.sheets()[0]          #通过索引顺序获取
    
    # get full-text tweets
    texts = []
    test_idx = 5 if lst == 'first_2w' else 6
    for i in range(1, table.nrows):
        texts.append(table.row(i)[test_idx].value)

    texts = pd.DataFrame(texts, columns=['text'])
    # texts.insert(1, 'places', pd.Series([[]]*len(texts)))
    
    # match all palces for all lines
    places = []
    sparse_matirx = []
    for i in tqdm(location_names.unique()): #[297604:]
        try:
            result = texts.text.str.contains(i)
            if result.to_numpy().sum() > 0:
                places.append(i)
                sparse_matirx.append(result.to_numpy())
        except Exception as e:
            continue
    
    # save match results with pickle
    print("出现的地点数: ", len(places), " 记录的列数:", len(sparse_matirx))
    with open('place2index_%s_green.pickle' % lst, 'wb') as f:
        pickle.dump([places, sparse_matirx], f)
        
    # print the most places in all tweets
    name2num = []
    for i,p in enumerate(places):
        name2num.append(sparse_matirx[i].sum())

    name2num = np.array(name2num)
    order_num = np.flip(np.argsort(name2num))
    np_places = np.array(places)
    numofprint = 5
    for p, n in zip(np_places[order_num][:numofprint], name2num[order_num][:numofprint]):
        print(p, ":", n)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 124/124 [00:00<00:00, 801.17it/s]


22 22
天津动物园 : 15
天津水上公园 : 15
滨海文化中心 : 11
水西公园 : 10
西沽公园 : 9


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 124/124 [00:00<00:00, 855.72it/s]

22 22
天津动物园 : 12
滨海文化中心 : 9
人民公园 : 7
天津文化中心 : 5
天津水上公园 : 4


In [ ]:
# sparse_matirx.shape

In [38]:
# read poi from shp files
shp_files = glob.glob('/home/yangwenhao/local/project/weibo_filter/app/data/tianjin/*.shp')

places_names_loc = []
for f in tqdm(shp_files):
    # print(f)
    x = geopandas.read_file(f, engine="pyogrio")
    places_names_loc.append(x[['name', 'geometry']])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:22<00:00,  1.04it/s]


In [32]:
print(x.geometry[0].x, x.geometry[0].y) 
# str(x.geometry[0])

116.7644123073674 38.82259674787439


'POINT (116.7644123073674 38.82259674787439)'

In [55]:
places_names_loc = pd.concat(places_names_loc)

names2loc = {}
for l in places_names_loc.iterrows():
    # print(l[1].values[0], l[1].values[1],)
    names2loc[l[1].values[0]] = l[1].values[1]

In [50]:
places_names_loc.head(2)

,name,geometry
0,元记电气焊补胎钣金,POINT (116.76441 38.82260)
1,元汇津通卸载,POINT (116.75977 38.82440)


In [ ]:
# read additional cols
with open('place2index.pickle', 'rb') as f:
    [places, sparse_matirx] = pickle.load(f)

places_np = np.array(places)
sparse_np = np.array(sparse_matirx) #.shape

append_cells = []
for l in range(sparse_np.shape[1]):
    row_cells = []
    for i in places_np[np.where(sparse_np[:, l] == True)[0]]:
        i_xy = names2loc[i]
        row_cells.append([i, i_xy.x, i_xy.y])
    append_cells.append(row_cells)

In [ ]:
append_cells[:10]

[[['酒店', 117.70100359480168, 39.296045987523456],
  ['天津', 117.17989775825995, 39.12978780211583],
  ['天津瑞吉金融街酒店', 117.19148461137928, 39.12858938331741],
  ['金融街', 117.20638076565724, 39.12375796277156]],
 [['饭', 117.54906841621506, 39.41817775960832]],
 [['小白楼', 117.20932596977684, 39.1159842943953],
  ['天津', 117.17989775825995, 39.12978780211583],
  ['白楼', 117.28157823731766, 39.47219240381378],
  ['西洋', 117.5147029707013, 39.03815707550861]],
 [['雾造', 116.91964343118586, 38.936991545703805],
  ['津南区', 117.35020535412332, 38.93594862947365],
  ['天津', 117.17989775825995, 39.12978780211583]],
 [['天津', 117.17989775825995, 39.12978780211583]],
 [['天津', 117.17989775825995, 39.12978780211583]],
 [['天津', 117.17989775825995, 39.12978780211583]],
 [['天津', 117.17989775825995, 39.12978780211583]],
 [['天津', 117.17989775825995, 39.12978780211583]],
 [['天津', 117.17989775825995, 39.12978780211583]]]

In [73]:
import xlrd
import xlwt
from xlutils.copy import copy

In [89]:
green_loc = set(location_names.to_list())

In [92]:
headers = ["相关地点数量", "微博id", "微博主页", "文本", "日期", "图片", "转发数", "评论数", "点赞数"]
read_cols = [2, 3, 5, 6, 7, 8, 9, 10]

for lst in ['first_2w']: #, 
    # read additional cols
    with open('place2index.pickle', 'rb') as f:
        [places, sparse_matirx] = pickle.load(f)

    places_np = np.array(places)
    sparse_np = np.array(sparse_matirx) #.shape

    append_cells = []
    for l in range(sparse_np.shape[1]):
        row_cells = []
        for i in places_np[np.where(sparse_np[:, l] == True)[0]]:
            i_xy = names2loc[i]
            row_cells.append([i, i_xy.x, i_xy.y])
        append_cells.append(row_cells)
    
    max_los = 1
    for i in append_cells:
        max_los = max(max_los, len(i))
    
    read_lst = r'/home/yangwenhao/local/project/weibo_filter/app/data/' + lst + '.xls'
    save_lst = r'/home/yangwenhao/local/project/weibo_filter/app/data/' + lst + '_loc.xls'

    read_writebook = xlrd.open_workbook(filename=read_lst)
    # writebook = copy(wb=read_writebook)
    # writesheet = writebook.get_sheet(0)
    
    writebook = xlwt.Workbook(encoding='utf-8')
    # 添加sheet 写入excel, 参数对应 行, 列, 值
    poi_sheet     = writebook.add_sheet(u'地点检索到公园POI',   cell_overwrite_ok=True)
    
    npoi_sheet    = writebook.add_sheet(u'地点检索到非公园POI', cell_overwrite_ok=True)
    tianjin_sheet = writebook.add_sheet(u'地点仅检索到天津',    cell_overwrite_ok=True)
    
    poi_row, npoi_row, tianjin_row = 1, 1, 1
    
# 相关地点数量	微博id	微博主页	文本	日期	图片	转发数	评论数	点赞数
    for sheet in [poi_sheet, npoi_sheet, tianjin_sheet]:
        for i, h in enumerate(headers):
            sheet.write(0, i, label=h)
    
    for sheet in [poi_sheet, npoi_sheet]:
        for i in range(1, max_los+1):
            sheet.write(0, (i-1)*3+len(headers),   label='地点提取%d'%i)
            sheet.write(0, (i-1)*3+len(headers)+1, label='经度%d'%i)
            sheet.write(0, (i-1)*3+len(headers)+2, label='纬度%d'%i)
    
    tianjin_sheet.write(0, len(headers), label='地点提取')
        
    read_table = read_writebook.sheet_by_index(0)
    for i in range(1, nrows):
        this_line = read_table.row_slice(i)
        this_locs = append_cells[i-1]
        locs_names = [l[0] for l in this_locs]
        
        green_tweet = False
        for l in locs_names:
            if l in green_loc:
                green_tweet = True
                break
                
        if len(locs_names) == 1 and locs_names[0] == '天津':
            writesheet = tianjin_sheet
            this_row = tianjin_row
            tianjin_row += 1
        elif green_tweet:
            writesheet = poi_sheet
            this_row = poi_row
            poi_row += 1
        else:
            writesheet = npoi_sheet
            this_row = npoi_row
            npoi_row += 1
        
        writesheet.write(this_row, 0, label=len(append_cells[i-1]))
                         
        for j, c in enumerate(read_cols):
            writesheet.write(this_row, j+1, label=this_line[c].value)
        
        for j, (name, x, y) in enumerate(append_cells[i-1]):
            writesheet.write(this_row, j*3+len(headers),   label=name)
            writesheet.write(this_row, j*3+len(headers)+1, label=x)
            writesheet.write(this_row, j*3+len(headers)+2, label=y)

    writebook.save(save_lst)

In [96]:
headers = ["相关地点数量", "微博id", "微博主页", "文本", "日期", "图片", "转发数", "评论数", "点赞数"]
read_cols = [2, 3, 6, 7, 8, 9, 10, 11]

for lst in ['noimg_6w']: #'first_2w']: #, 
    # read additional cols
    with open('place2index_%s.pickle'%(lst), 'rb') as f:
        [places, sparse_matirx] = pickle.load(f)

    places_np = np.array(places)
    sparse_np = np.array(sparse_matirx) #.shape

    append_cells = []
    for l in range(sparse_np.shape[1]):
        row_cells = []
        for i in places_np[np.where(sparse_np[:, l] == True)[0]]:
            i_xy = names2loc[i]
            row_cells.append([i, i_xy.x, i_xy.y])
        append_cells.append(row_cells)
    
    max_los = 1
    for i in append_cells:
        max_los = max(max_los, len(i))
    
    read_lst = r'/home/yangwenhao/local/project/weibo_filter/app/data/' + lst + '.xls'
    save_lst = r'/home/yangwenhao/local/project/weibo_filter/app/data/' + lst + '_loc.xls'

    
    # writebook = copy(wb=read_writebook)
    # writesheet = writebook.get_sheet(0)
    
    writebook = xlwt.Workbook(encoding='utf-8')
    # 添加sheet 写入excel, 参数对应 行, 列, 值
    poi_sheet     = writebook.add_sheet(u'地点检索到公园POI',   cell_overwrite_ok=True)
    npoi_sheet    = writebook.add_sheet(u'地点检索到非公园POI', cell_overwrite_ok=True)
    tianjin_sheet = writebook.add_sheet(u'地点仅检索到天津',    cell_overwrite_ok=True)
    
    poi_row, npoi_row, tianjin_row = 1, 1, 1
    
# 相关地点数量	微博id	微博主页	文本	日期	图片	转发数	评论数	点赞数
    for sheet in [poi_sheet, npoi_sheet, tianjin_sheet]:
        for i, h in enumerate(headers):
            sheet.write(0, i, label=h)
    
    for sheet in [poi_sheet, npoi_sheet]:
        for i in range(1, max_los+1):
            sheet.write(0, (i-1)*3+len(headers),   label='地点提取%d'%i)
            sheet.write(0, (i-1)*3+len(headers)+1, label='经度%d'%i)
            sheet.write(0, (i-1)*3+len(headers)+2, label='纬度%d'%i)
    
    tianjin_sheet.write(0, len(headers), label='地点提取')
    
    read_writebook = xlrd.open_workbook(filename=read_lst)
    read_table = read_writebook.sheet_by_index(0)
    nrows = read_table.nrows
    print("Num of rows: ", nrows)
    
    for i in range(1, nrows):
        this_line = read_table.row_slice(i)
        this_locs = append_cells[i-1]
        locs_names = [l[0] for l in this_locs]
        
        green_tweet = False
        for l in locs_names:
            if l in green_loc:
                green_tweet = True
                break
                
        if len(locs_names) == 1 and locs_names[0] == '天津':
            writesheet = tianjin_sheet
            this_row = tianjin_row
            tianjin_row += 1
        elif green_tweet:
            writesheet = poi_sheet
            this_row = poi_row
            poi_row += 1
        else:
            writesheet = npoi_sheet
            this_row = npoi_row
            npoi_row += 1
        
        writesheet.write(this_row, 0, label=len(append_cells[i-1]))
                                 
        for j, c in enumerate(read_cols):
            writesheet.write(this_row, j+1, label=this_line[c].value)
        
        for j, (name, x, y) in enumerate(append_cells[i-1]):
            writesheet.write(this_row, j*3+len(headers), label=name)
            writesheet.write(this_row, j*3+len(headers)+1, label=x)
            writesheet.write(this_row, j*3+len(headers)+2, label=y)

    writebook.save(save_lst)

Num of rows:  5540
